# Prerequisites

## Loading users_001.csv into DataFrame

In [0]:
df = spark.read.csv(path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/user_dataset/users_001.csv",
        header=True,
        inferSchema=True,
    )
df.display()

## Create Delta Table

In [0]:
%sql
USE CATALOG quickstart_catalog;

CREATE TABLE quickstart_schema.users_delta (
  id INT,
  name STRING,
  dob DATE,
  email STRING,
  gender STRING,
  country STRING,
  region STRING,
  city STRING,
  asset INT,
  marital_status STRING
);

DESCRIBE EXTENDED quickstart_schema.users_delta;

# Loading data into Delta Table - Transaction 02

In [0]:
df.write.mode("overwrite").saveAsTable("quickstart_schema.users_delta")

# Read Delta Table

In [0]:
spark.read.table("quickstart_schema.users_delta").display()

# Transaction History

## Using Delta Table API

In [0]:
from delta.tables import DeltaTable
 
delta_table = DeltaTable.forName(
    spark, "quickstart_schema.users_delta"
)
delta_table.history().display()

## Using SQL

In [0]:
DESCRIBE HISTORY quickstart_schema.users_delta

# Transaction 03

In [0]:
from pyspark.sql.functions import col
df.filter(col("country")=="India").write.mode("overwrite").saveAsTable("quickstart_schema.users_delta")

In [0]:
spark.read.table("quickstart_schema.users_delta").display()

# Read specific version

## SQL

In [0]:
%sql

SELECT * FROM quickstart_schema.users_delta VERSION AS OF 1 LIMIT 4;

## Python API

In [0]:
spark.read.option("versionAsOf", 2).table("quickstart_schema.users_delta").display()

# Time Travel

In [0]:
%sql
 
SELECT * from quickstart_catalog.quickstart_schema.users_delta TIMESTAMP AS OF '2025-11-27T13:45:29' LIMIT 10;